In [44]:
import os
import json

import urlparse
import datetime

class Bot(object):
    repository = "oo_json"
    filepath_results = NotImplemented

    def __init__(self, uri):
        self.dataset = []
        self.info = {}

        self.init()
        
        
        self.set_uri(uri)
        self.content = None
        self.results = []

    def init(self):
        if not os.path.exists(self.repository):
            os.makedirs(self.repository)
        else:
            print "Already found this folder - {}".format(self.repository)

    def process(self):
        self.crawl_results()
        self.parse_results()
        self.write_results()

    def set_uri(self, uri):
        self.uri = uri

    def crawl_results(self):
        if self.uri:
            response = urllib2.urlopen(self.uri)

            self.content = response.read()
        else:
            print "Pleaset set the value for self.uri"

    def parse_results(self):
        raise NotImplementedError
        
    def write_results(self):
        if self.results:
            filepath = os.path.join(self.repository, self.filepath_results)
            with open(filepath, "wb") as out_file:
                json.dump(self.results, out_file)
        else:
            print "Found the self.results is None"


# 抓取台灣銀行匯率資料

In [45]:
class FxRateBot(Bot):
    filepath_results = "fxrate.json"

    def parse_results(self):
        if self.content:
            is_header = True
            headers = []
            for line in self.content.split("\n"):
                if is_header:
                    headers = line.strip().split(",")

                    is_header = False
                else:
                    self.results.append(dict(zip(headers, line.strip().split(","))))
        else:
            print "Found the self.content is None"
            
uri = "http://rate.bot.com.tw/xrt/flcsv/0/day"
bot = FxRateBot(uri)
bot.process()

Already found this folder - oo_json


# 抓取台北市公車資料

In [48]:
class BusBot(Bot):
    filepath_results = "taipei_bus.json"
    
    def parse_results(self):
        pass
    
uri = "http://ptx.transportdata.tw/MOTC/v2/Bus/EstimatedTimeOfArrival/City/Taipei?$top=30000&$format=JSON"
bot = BusBot(uri)
bot.process()

Already found this folder - oo_json
Found the self.results is None


# 抓取台灣興櫃公司股票資訊

In [52]:
class EmergingStock(Bot):
    filepath_results = "emerging_stock.json"
    
    def crawl_results(self):
        super(EmergingStock, self).crawl_results()
        
        self.content = unicode(self.content, "big5")
    
    def parse_results(self):
        headers, results = [], []
        for idx, line in enumerate(self.content.split("\n")):
            if idx == 2:
                headers = line.strip().split(",")
            elif idx > 2:
                self.results.append(dict(zip(headers, line.strip().split(","))))
            else:
                pass
            
uri = "http://www.tpex.org.tw/storage/emgstk/ch/new.csv?t=1490626439859"
bot = EmergingStock(uri)
bot.process()

Already found this folder - oo_json
